In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
all_text = pd.read_csv('../data/creative_all_text.csv')
logo_info = pd.read_csv('../data/creative_logo_info.csv')
objects = pd.read_csv('../data/creative_objects.csv')
engagement_btn = pd.read_csv('../data/engagement_btn_position.csv')
cta_info = pd.read_csv('../data/cta_size_v3.csv')

In [3]:
temp1 = all_text.merge(logo_info,how='left',on='game_id')
temp1.drop(columns=["Unnamed: 0"],inplace=True)

temp2 = temp1.merge(objects,how='left',on='game_id')

temp2.drop(columns=['preview_link_y','ER_y','CTR_y'],inplace=True)
temp2.rename(columns={'preview_link_x':'preview_link','ER_x':'ER','CTR_x':'CTR'},inplace=True)

temp3 = temp2.merge(engagement_btn,how='left',on='game_id')
temp3.drop(columns='Unnamed: 0',inplace=True)

temp3.rename(columns={'file_name':'logo_file','preview':'preview_file'},inplace=True)

### clean up cta info before merging

In [4]:
def clean_rgb(txt:list):
    txt = txt[0]
    txt = txt.strip('][').split('\n')
    lst = list(map(lambda x:(re.sub(' +', ' ', x.strip(' ]['))).split(' '),txt))

    array = np.array(lst)
    array = array.astype(int)
    array = np.mean(array,axis=0).astype(int)
    
    return separate_rgb(array)

In [5]:
def separate_rgb(rgb:list):
    red = rgb[0]
    green = rgb[1]
    blue = rgb[2]
    return (red,green,blue)

In [6]:
# cta_info.head()

In [7]:
cta_info.drop(columns=['image','Unnamed: 0'],inplace=True)
cta_info.rename(columns={'label':'game_id'},inplace=True)
cta_info_grouped = cta_info.groupby(by='game_id',as_index=False)
cta_info_group_combined = cta_info_grouped.aggregate({'cta_text': lambda x: list(x),'cta_dominant_colors_in_rgb': lambda x: list(x),'cta_width':'mean','cta_height':'mean'})
# cta_info_group_combined_num = cta_info_grouped.aggregate(cta_width=('cta_width','mean'),cta_height=('cta_height','mean'))

In [8]:
cta_info_group_combined['cta_text']=cta_info_group_combined.cta_text.apply(lambda x: re.sub('[\[\'\"\],]', '', str(x)))

In [9]:
cta_info_group_combined[['red','green','blue']] = cta_info_group_combined.cta_dominant_colors_in_rgb.apply(lambda x:pd.Series(clean_rgb(x)))

In [10]:
cta_info_group_combined.drop(columns="cta_dominant_colors_in_rgb",inplace=True)

In [11]:
# cta_info_group_combined.head()

### merge all together

In [12]:
merged_df = temp3.merge(cta_info_group_combined,how='left',on='game_id')
merged_df.head()

,game_id,preview_link,ER,CTR,all_text,logo_file,shape,top_left,bottom_right,LAR,...,engagement_instraction,preview_file,eng_top_left_pos,eng_bottom_right_pos,cta_text,cta_width,cta_height,red,green,blue
0,bfbf7a2b7ac635e67877b1ab87fd6629,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.209269,0.058438,"['PHILADELPHIA AREA', '1EXUS DEALERS', 'Ovexus...",NaN,NaN,NaN,NaN,NaN,...,engagement_instruction.png,_preview.png,"(60, 185)","(526, 262)",LEARN MORE,64.0,300.0,23.0,37.0,25.0
1,945010afbf9a55bbdffcb0895f946155,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.274552,0.074731,"['Ovexus', 'L/CERTIFIED BY LEXUS', 'PHILADELPH...",NaN,NaN,NaN,NaN,NaN,...,engagement_instruction.png,_preview.png,"(60, 185)","(526, 262)",LEARN MORE,64.0,300.0,23.0,37.0,25.0
2,e68e20f592457b875ce29757ab855dfe,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.103688,0.042228,"['PHILADELPHIA AREA', '1 EXUS DEALERS', 'Ovexu...",NaN,NaN,NaN,NaN,NaN,...,engagement_instruction.png,_preview.png,"(50, 114)","(430, 177)",LEARN MORE,50.0,234.0,23.0,37.0,25.0
3,adunit-nestle-purina-friskies-mob,https://s3.ap-southeast-1.amazonaws.com/a.futu...,0.136963,0.005393,"['(= POR OOK', 'aU', ' ', '2025 PACKAGING', ...",['logo.png'],"(859, 627)","(182, 580)","(426, 754)",0.078828,...,NaN,NaN,NaN,NaN,,79.0,227.0,149.0,78.0,65.0
4,adunit-lionsgate-uwomt-user-slider-sensory-vid...,https://s3.us-west-1.amazonaws.com/a.futureadl...,0.114208,0.016443,"['N THEATERS FUERYWHERPRIL 2024', 'Te', 'SUS',...",['logo.png'],"(500, 600)","(126, 309)","(449, 424)",0.123817,...,NaN,NaN,NaN,NaN,Te,60.0,206.0,128.0,191.0,254.0


In [13]:

def width_height(row,top_left_col:str,bottom_right_col:str):
    
    top_left = row[top_left_col].strip(')(').split(',')
    x1 = top_left[0]
    y1 = top_left[1]
   
    bottom_right = row[bottom_right_col].strip(')(').split(',')

    x2 = bottom_right[0]
    y2 = bottom_right[1]

    height = int(y2) - int(y1)
    width = int(x2) - int(x1)

    return (width,height)

def width_height_image(shape):
    
    try:
        width_height = shape.strip(')(').split(',')
        width = width_height[0]
        height = width_height[1]
    except:
        return (0,0)
   
    return (width,height)

In [14]:
merged_df[['top_left','bottom_right','eng_top_left_pos','eng_bottom_right_pos']]=merged_df[['top_left','bottom_right','eng_top_left_pos','eng_bottom_right_pos']].fillna('(0,0)')
merged_df[['LAR']]=merged_df[['LAR']].fillna(0)

merged_df['all_text'] = merged_df.all_text.apply(lambda x: x.strip(' ][').replace(',','').replace('\'',''))
merged_df[['logo_width','logo_height']]=merged_df.apply(lambda x:pd.Series(width_height(x,"top_left","bottom_right")),axis=1)
merged_df[['eng_width','eng_height']]=merged_df.apply(lambda x:pd.Series(width_height(x,"eng_top_left_pos","eng_bottom_right_pos")),axis=1)
merged_df[['preview_width','preview_height']]=merged_df['shape'].apply(lambda x:pd.Series(width_height_image(x)))

merged_df.drop(columns=['top_left','bottom_right','eng_top_left_pos','eng_bottom_right_pos','shape'],inplace=True)

In [15]:
useful_features = merged_df.copy()
useful_features.drop(columns=['game_id','preview_link','all_objects','logo_file','engagement_instraction','preview_file'],inplace=True)

In [ ]:
useful_features.info()

In [16]:
useful_features.head()

,ER,CTR,all_text,LAR,all_objects_count,unique_objects_count,cta_text,cta_width,cta_height,red,green,blue,logo_width,logo_height,eng_width,eng_height,preview_width,preview_height
0,0.209269,0.058438,PHILADELPHIA AREA 1EXUS DEALERS Ovexus L/CERTI...,0.000000,9,2,LEARN MORE,64.0,300.0,23.0,37.0,25.0,0,0,466,77,0,0
1,0.274552,0.074731,Ovexus L/CERTIFIED BY LEXUS PHILADELPHIA AREA ...,0.000000,13,2,LEARN MORE,64.0,300.0,23.0,37.0,25.0,0,0,466,77,0,0
2,0.103688,0.042228,PHILADELPHIA AREA 1 EXUS DEALERS Ovexus L/CERT...,0.000000,11,2,LEARN MORE,50.0,234.0,23.0,37.0,25.0,0,0,380,63,0,0
3,0.136963,0.005393,(= POR OOK aU 2025 PACKAGING COMMITMENT FE...,0.078828,6,4,,79.0,227.0,149.0,78.0,65.0,244,174,0,0,859,627
4,0.114208,0.016443,N THEATERS FUERYWHERPRIL 2024 Te SUS THE NEW A...,0.123817,8,1,Te,60.0,206.0,128.0,191.0,254.0,323,115,0,0,500,600


In [18]:
useful_features.to_csv('../junk/cleaned_df.csv',index=False)